## Receiving directionality

(acoustics-receiver_plane_wave_approx)=
### Plane wave approximation

DISCUSSION PLANE WAVE APPROXIMATION!


### Directionality

While we often refer to hydrophones as points in the ocean, they all have a physical shape and size. This means that sound is not captured at a single point, but rather across a collection of sound-transducing elements on the hydrophone. As a result, every hydrophone has a certain _directionality_, or _beampattern_, which describes how the received sound level varies depending on the direction of incoming sound relative to the hydrophone. Depending on the application, sometimes we want an "omnidirectional" hydrophone that receives sound equally from all directions; at other times, we may want hydrophones that have a strong directionality, such that we can capture sound from a specific direction more clearly. 

Directionality arises because sound waves arriving at different parts of a receiver capture slightly different versions of the same sound. When these signals are combined, they create constructive and destructive interference depending on the sound direction.

We can see this clearly using widget below, showing signals received by each of the two elements of a receiver _array_ and their combined output. Depending where the direction of incoming sound and its frequency, the differences between the two signals can significantly affect the amplitude of their sum, giving rise to the directionality.

In [1]:
# ADD WIDGET
# - FIX 2 ELEMENTS
# - VARY: SOUND FREQUENCY
# - VARY: SOUND SOURCE DIRECTION: PLOT ON THE RIGHT AS AN ARROW
# - PLOT 2 RECEIVED SIGNALS ON THE LEFT OF ELEMENT
# - PLOT THE SUM OF THE SIGNALS BELOW THE INDIVIDUAL SIGNAL
# - PLOT A VERTICAL GRAPH ON THE RIGHT SHOWING RMS OR PEAK AMPLITUDE VS ANGLE, WITH A POINT INDICATING THE CURRENT SOURCE DIRECTION


In [2]:
import matplotlib
matplotlib.use('module://matplotlib_inline.backend_inline')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets

from IPython.display import display

In [3]:
def update_2_rcvr(angle, freq):
    
    angle_rad = np.deg2rad(angle)
    r = 1  # viz source circle radius
    d = 0.5  # element spacing
    c = 1500  # sound speed
    t = np.linspace(0, 0.01, 1000)
    y = np.sin(2 * np.pi * freq * t)
    phase_shift = d * np.sin(angle_rad) / (c/freq) * 2 * np.pi
    y1 = np.sin(2 * np.pi * freq * t - phase_shift)

    # beampattern across angle
    angle_all = np.arange(-90, 91, 1)
    angle_rad_all = np.deg2rad(angle_all)
    phase_shift_all = d * np.sin(angle_rad_all) / (c/freq) * 2 * np.pi
    y = np.sin(2 * np.pi * freq * t)
    y1_all = np.sin(np.tile(2 * np.pi * freq * t, (phase_shift_all.size,1))
                    - np.tile(phase_shift_all, (t.size, 1)).T)
    y_sum_all = (y + y1_all).max(axis=1)

    # PLOTS
    fig = plt.figure(figsize=(12, 3.5))
    gs = gridspec.GridSpec(1, 3, width_ratios=[4, 4, 2], wspace=0.3) 
    gs_signals = gridspec.GridSpecFromSubplotSpec(
        2, 1, height_ratios=[2,1], subplot_spec=gs[0, 1], hspace=0.3)

    # elements and source angle
    map_ax = plt.subplot(gs[0, 0])
    map_ax.plot(r*np.cos(angle_rad), r*np.sin(angle_rad), "ro",
                markerfacecolor="none", label="Source")
    map_ax.plot(0, d/10, "C0o", markersize=3)
    map_ax.plot(0, -d/10, "C1o", markersize=3)
    map_ax.set_xlim(-1.2, 1.2)
    map_ax.set_ylim(-1.2, 1.2)
    map_ax.axis("off")
    # map_ax.axis("equal")

    # individual received signal
    indiv_ax = plt.subplot(gs_signals[0])
    indiv_ax.plot(t*1e3, y-0.5, "C0", label="#1")
    indiv_ax.plot(t*1e3, y1+0.5, "C1", label="#2")
    indiv_ax.set_xlim(0, 10)
    indiv_ax.set_ylim(-2, 2.8)
    indiv_ax.set_xticklabels("")
    indiv_ax.legend(ncol=2, fontsize=9)
    indiv_ax.set_yticks([])
    indiv_ax.set_ylabel("Voltage")
    indiv_ax.set_title("Individual signal")

    # summed received signal
    sum_ax = plt.subplot(gs_signals[1])
    sum_ax.plot(t*1e3, y+y1, "k")
    sum_ax.set_xlim(0, 10)
    sum_ax.set_ylim(-2.2, 2.2)
    sum_ax.set_yticks([])
    sum_ax.set_ylabel("Voltage")
    sum_ax.set_title("Summed signal")

    # beampattern
    gs_beam_pattern = gridspec.GridSpecFromSubplotSpec(
        1, 1, subplot_spec=gs[0, 2], hspace=0.8)
    bp_ax = plt.subplot(gs_beam_pattern[0])
    bp_ax.plot(y_sum_all, angle_all, "k")
    bp_ax.plot((y+y1).max(), angle, "ro", markerfacecolor="w")
    bp_ax.set_xlim(0, 2.1)
    bp_ax.set_ylim(-90, 90)
    bp_ax.set_yticks(np.arange(-90, 91, 30))
    bp_ax.set_ylabel("Angle")
    bp_ax.set_title("Amplitude of sum")

    plt.show()


angle_slider = widgets.IntSlider(
    value=0, min=-90, max=90, step=1,
    description="Angle (°)",
    continuous_update=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="400px")
)

freq_slider = widgets.IntSlider(
    value=1000, min=100, max=2000, step=1, 
    description="Frequency (Hz)", 
    continuous_update=True, 
    style={'description_width': 'initial'}, 
    layout=widgets.Layout(width="400px")
)

interactive_plot = widgets.interactive(
    update_2_rcvr, angle=angle_slider, freq=freq_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='Angle (°)', layout=Layout(width='400px'), max=90, min=-9…

The directionality also changes depending on the number of elements and their spacing in the array:


In [4]:

def update_N_rcvr(angle, freq, N, d):
    
    angle_rad = np.deg2rad(angle)
    r = 1  # viz source circle radius
    c = 1500  # sound speed
    t = np.linspace(0, 0.01, 1000)
    y = np.sin(2 * np.pi * freq * t)
    y1 = np.zeros((t.size, N))
    for nn in np.arange(1, N, 1):
        phase_shift = nn * d * np.sin(angle_rad) / (c/freq) * 2 * np.pi
        y1[:, nn] = np.sin(2 * np.pi * freq * t - phase_shift)
    y1[:, 0] = y

    # beampattern across angle
    angle_all = np.arange(-90, 91, 1)
    angle_rad_all = np.deg2rad(angle_all)
    y = np.sin(2 * np.pi * freq * t)
    y1_all = np.zeros((angle_all.size, t.size, N))
    for nn in np.arange(1, N, 1):
        phase_shift_all = nn * d * np.sin(angle_rad_all) / (c/freq) * 2 * np.pi
        y1_all[:, :, nn] = np.sin(np.tile(2 * np.pi * freq * t, (phase_shift_all.size,1))
                        - np.tile(phase_shift_all, (t.size, 1)).T)
    y1_all[:, :, 0] = y
    y_sum_all = y1_all.sum(axis=-1).max(axis=1)

    # PLOTS
    fig = plt.figure(figsize=(12, 3.5))
    gs = gridspec.GridSpec(1, 3, width_ratios=[4, 4, 2], wspace=0.3) 
    gs_signals = gridspec.GridSpecFromSubplotSpec(
        2, 1, height_ratios=[2,1], subplot_spec=gs[0, 1], hspace=0.3)

    # elements and source angle
    map_ax = plt.subplot(gs[0, 0])
    map_ax.plot(r*np.cos(angle_rad), r*np.sin(angle_rad), "ro",
                markerfacecolor="none", label="Source")
    [map_ax.plot(0, d/10*nn - N/2*d/10, f"C{nn}o", markersize=3) for nn in np.arange(N)]
    map_ax.set_xlim(-1.2, 1.2)
    map_ax.set_ylim(-1.2, 1.2)
    map_ax.axis("off")
    # map_ax.axis("equal")

    # individual received signal
    indiv_ax = plt.subplot(gs_signals[0])
    [indiv_ax.plot(t*1e3, y1[:, nn]+0.5*nn, f"C{nn}", label=f"#{nn}") for nn in np.arange(N)]
    indiv_ax.set_xlim(0, 10)
    # indiv_ax.set_ylim(-2, 2.8)
    indiv_ax.set_xticklabels("")
    indiv_ax.legend(ncol=4, fontsize=8, loc="upper right")
    indiv_ax.set_yticks([])
    indiv_ax.set_ylabel("Voltage")
    indiv_ax.set_title("Individual signal")

    # summed received signal
    sum_ax = plt.subplot(gs_signals[1])
    sum_ax.plot(t*1e3, y1.sum(axis=1), "k")
    sum_ax.set_xlim(0, 10)
    sum_ax.set_ylim(-N-0.2, N+0.2)
    sum_ax.set_yticks([])
    sum_ax.set_ylabel("Voltage")
    sum_ax.set_title("Summed signal")

    # beampattern
    gs_beam_pattern = gridspec.GridSpecFromSubplotSpec(
        1, 1, subplot_spec=gs[0, 2], hspace=0.8)
    bp_ax = plt.subplot(gs_beam_pattern[0])
    bp_ax.plot(y_sum_all, angle_all, "k")
    bp_ax.plot(y1.sum(axis=1).max(), angle, "ro", markerfacecolor="w")
    bp_ax.set_xlim(0, N+0.1)
    bp_ax.set_ylim(-90, 90)
    bp_ax.set_yticks(np.arange(-90, 91, 30))
    bp_ax.set_ylabel("Angle")
    bp_ax.set_title("Amplitude of sum")

    plt.show()


angle_slider = widgets.IntSlider(
    value=0, min=-90, max=90, step=1,
    description="Angle (°)",
    continuous_update=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="400px")
)

freq_slider = widgets.IntSlider(
    value=1000, min=100, max=2000, step=1, 
    description="Frequency (Hz)", 
    continuous_update=True, 
    style={'description_width': 'initial'}, 
    layout=widgets.Layout(width="400px")
)

N_slider = widgets.IntSlider(
    value=3, min=2, max=10, step=1, 
    description="# receivers", 
    continuous_update=True, 
    style={'description_width': 'initial'}, 
    layout=widgets.Layout(width="400px")
)

spacing_slider = widgets.FloatSlider(
    value=0.5, min=0.1, max=1, step=0.05, 
    description="Spacing (m)", continuous_update=True, 
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="400px")
)


interactive_plot = widgets.interactive(
    update_N_rcvr, 
    angle=angle_slider, freq=freq_slider, 
    N=N_slider, d=spacing_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='Angle (°)', layout=Layout(width='400px'), max=90, min=-9…

In [5]:
# HERE WE CAN EASILY LINK TO IN-DEPTH MATERIALS TO EXPLAIN MORE USING EQUATIONS

```{Tip}
:class: tip
Now you have the intuition of where receiving directionality comes about and what happens when the number of receiving elements changes. You will soon see how the same concept applies on [acoustic sources](acoustics-source)!
```